In [23]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import pandas as pd

C:\Users\tmhab\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: divide by zero encountered in add
  return f(*args, **kwds)
C:\Users\tmhab\anaconda3\lib\site-packages\pandas\tseries\offsets.py:2547: RuntimeWarning: divide by zero encountered in add
  _inc = Timedelta(microseconds=1000)
C:\Users\tmhab\anaconda3\lib\site-packages\pandas\tseries\offsets.py:2679: RuntimeWarning: divide by zero encountered in add
  _inc = Timedelta(days=1)
C:\Users\tmhab\anaconda3\lib\site-packages\pandas\tseries\offsets.py:2684: RuntimeWarning: divide by zero encountered in add
  _inc = Timedelta(hours=1)
C:\Users\tmhab\anaconda3\lib\site-packages\pandas\tseries\offsets.py:2689: RuntimeWarning: divide by zero encountered in add
  _inc = Timedelta(minutes=1)
C:\Users\tmhab\anaconda3\lib\site-packages\pandas\tseries\offsets.py:2694: RuntimeWarning: divide by zero encountered in add
  _inc = Timedelta(seconds=1)
C:\Users\tmhab\anaconda3\lib\site-packages\pandas\tseries\offsets.py:2699: RuntimeWarnin

## Import Files From Simulation:

In [2]:
file = "C:\\users/tmhab/wlcsim_membrane/data/2.10.21_NoInteractions_2hrs_r11v0.txt"
data = np.loadtxt(file) #D:\\myfiles\welcome.txt


## Our Approach to Calculating Percolation Paths Through a Polymer Membrane:

### Defining The System:

In our system we have random copolymers with solvent molecules. Polymers are represented by beads of types 1 and 2 and solvent molecules are individual beads of type 0. No bead can overlap another. 

### Defining Percolation:

A percolation is a pathway through the polymer network that solvent molecules can move through. To form a percolation a solvent molecule will need to have at least one other solvent molecule adjacent to it within some path radius $r$

### What are we trying to learn from the system:

The goal in developing this code is to determine quantitatively the amount of percolation in a membrane. 

### Models for Percolation:
**1. Fire Spread Model:** In this model we measure the percolation paths by tracing the spots where the solvent molecules continue to be adjacent to the previous... It's easier to explain with a picture

## What we need to calculate:

Starting with the final equilibrium state we have x,y,z postion of a bead and it's identity (and also methylation state). We want to calculate percolation paths which will be represented by root mean square. 

#### What Percolation Model do we want to use:

1. Fire spead model:

Proceedure:
1. Pick a bead type
2. pick a random bead of that bead type
3. Check surround bead types to see if they are the same
    a. Pick a radius to search in


In [3]:
def generate_test_data(length=10,width=10,height=10,datapoints=50):
    X = np.random.random(datapoints)*length
    Y = np.random.random(datapoints)*width
    Z = np.random.random(datapoints)*height
    Beads = np.random.randint(3,size=(datapoints)).astype('uint8')
    Beads
    return X,Y,Z,Beads

generate_test_data()

C:\Users\tmhab\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in multiply
  
C:\Users\tmhab\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in multiply
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\tmhab\anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in multiply
  after removing the cwd from sys.path.
C:\Users\tmhab\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:90: RuntimeWarning: divide by zero encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\tmhab\anaconda3\lib\site-packages\numpy\core\arrayprint.py:885: RuntimeWarning: divide by zero encountered in isfinite
  finite_vals = data[isfinite(data)]
C:\Users\tmhab\anaconda3\lib\site-packages\numpy\core\arrayprint.py:888: RuntimeWarning: divide by zero encountered in not_equal
  abs_non_zero = absolute(f

(array([9.60690521, 2.57654843, 9.76846541, 2.17424356, 7.93952997,
        7.31563887, 5.0555424 , 2.45383638, 8.0243156 , 0.82638458,
        8.49558114, 8.93988722, 5.29061312, 6.37212815, 9.29719126,
        8.73574779, 8.76370704, 4.3963401 , 5.99235968, 6.39966758,
        6.8597524 , 5.99759523, 2.22530451, 3.96729175, 6.9845253 ,
        2.54140316, 9.02190064, 1.20604032, 5.03479539, 2.30358726,
        6.37256655, 6.70112783, 7.43043419, 7.14929504, 4.82717904,
        4.92628296, 6.7544717 , 9.21099036, 3.76250581, 2.7844163 ,
        7.57603049, 8.14252238, 0.73694223, 1.69484761, 8.5600752 ,
        4.10860418, 0.23037294, 0.57008199, 0.05728708, 8.1853997 ]),
 array([0.94359528, 2.95359315, 3.11340401, 2.56484244, 4.73534713,
        4.77712817, 9.21386356, 4.02815455, 6.19557789, 0.58536096,
        9.35515799, 3.68851418, 1.684745  , 2.62268489, 5.62132229,
        0.19342031, 2.44413379, 0.42387444, 9.293501  , 0.01350537,
        7.35306608, 2.17984823, 6.32364764, 7.

In [4]:
class Bead:
    def __init__(self, x, y, z, bead_type):
        self.x = x
        self.y = y
        self.z = z
        self.bead_type = bead_type
        


In [5]:
class PolymerNetwork:
    """
    This represents the postions of bead and their type (time invariant)
    
    Parameters
    ----------
    X : float [array] X 
    """
    
    def __init__(self,X,Y,Z, Beads, bead_list=[]):
        self.X = X
        self.Y = Y
        self.Z = Z
        self.Beads = Beads
        self.bead_list = bead_list
        
        self.density = Beads.shape[0]/((max(X)-min(X)) * (max(Y)-min(Y)) * (max(Z)-min(Z)))
        
    def visualize(self):
        fig = plt.figure()
        ax = plt.axes(projection='3d')
        ax.scatter3D(self.X[Beads==0], self.Y[Beads==0], self.Z[Beads==0],color='y')
        ax.scatter3D(self.X[Beads==1], self.Y[Beads==1], self.Z[Beads==1],color='r')
        ax.scatter3D(self.X[Beads==2], self.Y[Beads==2], self.Z[Beads==2],color='b')
        
    def nearest_neighbor(self,bead_num,radius):
        '''
        Find the average number of beads that surround a bead of the same type within the radius specified
        '''
        X_bead = self.X[Beads==bead_num]
        Y_bead = self.Y[Beads==bead_num]
        Z_bead = self.Z[Beads==bead_num]
        
        total_neighbors = 0
        for i in range(Beads[Beads==bead_num].shape[0]):
            point = np.array([X_bead[i],Y_bead[i],Z_bead[i]])
            displacement = np.sqrt((X_bead - point[0])**2 + (Y_bead - point[1])**2 + (Z_bead - point[2])**2)
            num_neighbor_points = displacement[displacement<=radius].shape[0] -1
            
            total_neighbors += num_neighbor_points

        return total_neighbors/(i+1)
    def create_beads(self):
        if len(self.bead_list) != 0:
            raise Exception("bead list already populated")
        for i in range(len(self.X)):
            bead = Bead(self.X[i], self.Y[i], self.Z[i], self.Beads[i])
            self.bead_list.append(bead)
    
    def print_bead_list(self):
        for i, bead in enumerate(self.bead_list):
            print("Bead " + str(i) + ": " + str(bead.x) + ", " + str(bead.y) + ", " + str(bead.z) + " Type: " + str(bead.bead_type))


In [6]:
X,Y,Z,Beads = generate_test_data()
#PNet_Test = PolymerNetwork(X,Y,Z,Beads)

#PNet_Test.density

C:\Users\tmhab\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in multiply
  
C:\Users\tmhab\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in multiply
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\tmhab\anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in multiply
  after removing the cwd from sys.path.


In [7]:
#PNet_Test.visualize()

In [8]:
#PNet_Test.nearest_neighbor(bead_num=0,radius=0.4)

In [9]:

#PNet_Test.create_beads()
#PNet_Test.print_bead_list()


In [10]:
#radii = np.linspace(0,15,100)
#for r in radii:
#    plt.plot(r,PNet_Test.nearest_neighbor(bead_num=0,radius=r),'y.')
#plt.xlabel('Radius')
#plt.ylabel('Average Number of Neighbor Points')

In [11]:
'''
Determine the mean square interbead distance
Input: PolymerNetwork
Return: mean square distance b/w beads (?? issue w/ beads not on polymer chains)
TODO: convert to pd dataframes? may not need to once working w/ chains
TODO: change to using polymer chains, not just random beads
'''
def r2(polymer_network):
    number_beads = len(polymer_network.bead_list)
    bead_dist = np.zeros((number_beads, number_beads))
    for i, bead1 in enumerate(polymer_network.bead_list):
        for j, bead2 in enumerate(polymer_network.bead_list):
            position1 = np.array([bead1.x, bead1.y, bead1.z])
            position2 = np.array([bead2.x, bead2.y, bead2.z])
            displacement_squared = np.sum((position2 - position1)**2)
            bead_dist[i][j] = displacement_squared
            
    #print(bead_dist)
    #get diagonals, average them
    num_diagonals = (number_beads * 2) - 1
    n_jump_dist_avg = np.zeros(num_diagonals)
    for d in range(num_diagonals):
        diag = np.diag(bead_dist, d)
        n_jump_dist_avg[d] = np.average(diag)    # populated with average displacements of d jumps
    avg_r2_diagonal = np.average(n_jump_dist_avg)

    return avg_r2_diagonal
    

In [12]:
#x = r2(PNet_Test)
#print(x)

In [13]:
beads = np.zeros(len(data[:, 0]))   #no chemical identity data
no_interactions = PolymerNetwork(data[:, 0], data[:, 1], data[:, 2], beads)

C:\Users\tmhab\anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars


In [15]:
no_interactions.create_beads()
#no_interactions.print_bead_list()
len(no_interactions.bead_list)  # should be 80,000

80000

In [20]:
#no_interactions.visualize()  #too many beads?

In [21]:
#r2(no_interactions)

ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

In [27]:
np.zeros((80000, 80000))

ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.